In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.set_printoptions(linewidth=200, threshold=100000)
torch.cuda.is_available()

True

In [2]:
from src.vlstm_fwbw_v0.interface import vlstm_fw_torch, vlstm_fwbw_cuda
from src.vlstm_fwbw_v0.interface import vlstm_fw_cuda
from vlstm_formulations.vlstm_parallel import vlstm_parallel_fw_torch
# from vlstm_formulations.vlstm_parallel_tiled import vlstm_parallel_tiled
from src.vlstm_fwbw_v0.torch_impl import vlstm_fw_tiled_torch

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt220cu121/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt220cu121/lib


Using /home/max/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu121/vlstm_fwbw_v0/build.ninja...
Building extension module vlstm_fwbw_v0...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] /home/max/miniconda3/envs/xlstmpt220cu121/bin/nvcc --generate-dependencies-with-compile --dependency-output kernel_bw.cuda.o.d -ccbin /home/max/miniconda3/envs/xlstmpt220cu121/bin/x86_64-conda-linux-gnu-cc -DTORCH_EXTENSION_NAME=vlstm_fwbw_v0 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC -isystem /home/max/miniconda3/envs/xlstmpt220cu121/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERS

Loading extension module vlstm_fwbw_v0...


## CUDA vLSTM forward

In [42]:
S = 64 #32 #16 #8 # seq len
B = 2 # batch size
NH = 1 # num heads
DH = 8 # dim per head
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [43]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
# fixed:
qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.

# random: 
# qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
# ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
# vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
# igs = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S, 1) / 10.
# igs = torch.zeros((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
igs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
fgs = torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
# fgs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
qs, qs.shape, len(qs.view(-1))

(tensor([[[[0.0000e+00, 1.0000e-01, 2.0000e-01, 3.0000e-01, 4.0000e-01, 5.0000e-01, 6.0000e-01, 7.0000e-01],
           [8.0000e-01, 9.0000e-01, 1.0000e+00, 1.1000e+00, 1.2000e+00, 1.3000e+00, 1.4000e+00, 1.5000e+00],
           [1.6000e+00, 1.7000e+00, 1.8000e+00, 1.9000e+00, 2.0000e+00, 2.1000e+00, 2.2000e+00, 2.3000e+00],
           [2.4000e+00, 2.5000e+00, 2.6000e+00, 2.7000e+00, 2.8000e+00, 2.9000e+00, 3.0000e+00, 3.1000e+00],
           [3.2000e+00, 3.3000e+00, 3.4000e+00, 3.5000e+00, 3.6000e+00, 3.7000e+00, 3.8000e+00, 3.9000e+00],
           [4.0000e+00, 4.1000e+00, 4.2000e+00, 4.3000e+00, 4.4000e+00, 4.5000e+00, 4.6000e+00, 4.7000e+00],
           [4.8000e+00, 4.9000e+00, 5.0000e+00, 5.1000e+00, 5.2000e+00, 5.3000e+00, 5.4000e+00, 5.5000e+00],
           [5.6000e+00, 5.7000e+00, 5.8000e+00, 5.9000e+00, 6.0000e+00, 6.1000e+00, 6.2000e+00, 6.3000e+00],
           [6.4000e+00, 6.5000e+00, 6.6000e+00, 6.7000e+00, 6.8000e+00, 6.9000e+00, 7.0000e+00, 7.1000e+00],
           [7.2000e

In [44]:
hs_pt, n_pt, m_pt, _, matD_pt = vlstm_fw_torch(queries=qs, keys=ks, values=vs, igate_preact=igs, fgate_preact=fgs)

In [45]:
# result_gt, m, l, n = vlstm_fw_tiled_torch(queries=qs, keys=ks, values=vs, igate_preact=igs, fgate_preact=fgs, bq_tile_size=8, bkv_tile_size=8)
# result_gt, result_gt.shape, torch.exp(-m), l, n

In [47]:
# torch.nn.functional.logsigmoid(fgs).squeeze()[17] #+ (10.706538)
# (fgs).squeeze()[17] #+ (10.706538)

In [48]:
# cuda kernel
hs_cuda, n_cuda, m_cuda, matD_cuda = vlstm_fw_cuda(mat_Q=qs, mat_K=ks, mat_V=vs, igate_preact=igs.squeeze(-1), fgate_preact=fgs.squeeze(-1))

before kernel dispatch - float32!
B: 2, NH: 1, S: 64, DH: 8
blocksxy: 2-2, threadsxy: 4-4, shared_mem in bytes: 5664
In FW-Kernel: gdim.x: 2, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In FW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4
BIdx.y=0, FLTIdx=7: qTileIdx=0, fChunkAccIterIdx=0 (<1), fWCIdx=0, f_acc=-2.192832, fTileColLast=0.000000
STR: BIdx.y=1: qTileIdx=0, fWCIdx=0 (<1), flatThreadIdx=7: fT_acc_res=-4.698925, fTileCol_val=-4.698925, fTileColLastVal=0.000000
BIdx.y=0, FLTIdx=7: qTileIdx=0, fChunkAccIterIdx=1 (<1), fWCIdx=0, f_acc=-2.192832, fTileColLast=0.000000
BIdx.y=0, FLTIdx=7: qTileIdx=1, fChunkAccIterIdx=0 (<3), fWCIdx=0, f_acc=-2.506094, fTileColLast=-4.698925
STR: BIdx.y=1: qTileIdx=1, fWCIdx=0 (<1), flatThreadIdx=7: fT_acc_res=-9.711113, fTileCol_val=-9.711113, fTileColLastVal=-4.698925
BIdx.y=0, FLTIdx=7: qTileIdx=1, fChunkAccIterIdx=1 (<3), fWCIdx=0, f_acc=-2.506094, fTileColLast=-4.698925
BIdx.y=0, FLTIdx=7: qTileIdx=2, fChunkAccIterIdx=0 (<5), fWCIdx=0, f_acc=-2

In [49]:
RTOL = 1e-10
ATOL = 4e-4
print(f"hs match: {torch.allclose(hs_cuda, hs_pt, rtol=RTOL, atol=ATOL)}")
print(f"n match: {torch.allclose(n_cuda.unsqueeze(-1), n_pt, rtol=RTOL, atol=ATOL)}")
print(f"m match: {torch.allclose(m_cuda.unsqueeze(-1), m_pt, rtol=RTOL, atol=ATOL)}")
print(f"D match: {torch.allclose((matD_cuda - matD_pt).tril(), torch.zeros_like((matD_cuda)), rtol=RTOL, atol=ATOL)}")
# print(f"delta FBar match: {torch.allclose(dFgs_cu.unsqueeze(-1), dFgs_pt, rtol=RTOL, atol=ATOL)}")

hs match: True
n match: True
m match: True
D match: True


In [50]:
(matD_cuda - matD_pt).tril().abs().max()

tensor(1.4305e-05, device='cuda:0')

In [10]:
matD_pt[:, :, 0:64, :16]

tensor([[[[ 3.5158e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf],
          [ 3.8321e-02, -1.4171e+00,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf],
          [-2.7494e-01, -1.7304e+00, -8.3489e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf],
          [-5.8820e-01, -2.0437e+00, -1.1481e+00,  3.0957e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf],
          [-9.0146e-01, -2.3569e+00, -1.4614e+00, -3.6913e-03, -8.1536e-01, 

In [11]:
(matD_cuda - matD_pt)[:, :, 0:64, -10:]

tensor([[[[        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
          [        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
          [        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
          [        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
          [        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
          [        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
          [        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
      

In [12]:
hs_cuda - hs_pt

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00, -2.9802e-08,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 1.1921e-07, -5.9605e-08,  1.1921e-07, -8.9407e-08, -1.1921e-07, -1.4901e-08,  2.3842e-07, -1.1921e-07],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -5.9605e-08,  0.0000e+00],
          [-6.7055e-08,  0.0000e+00, -2.9802e-08, -1.7881e-07,  3.7253e-08,  4.4703e-08,  0.0000e+00,  1.0431e-07],
          [ 1.1921e-07,  0.0000e+00,  1.1921e-07,  8.9407e-08,  2.3842e-07, -5.9605e-08, -4.2492e-08,  1.1921e-07],
          [ 2.9802e-08,  2.9802e-08,  0.0000e+00, -1.1921e-07, -4.7684e-07,  2.3842e-07, -3.3528e-08,  8.9407e-08],
          [ 1.1921e-07,  0.0000e+00,  0.0000e+00, -8.9407e-08,  1.1921e-07,  5.9605e-08,  0.0000e+00,  6.7055e-08],
          [ 1.1921e-07, -5.9605e-08,  0.0000e+00,  1.1921e-07, -5.9605e-

In [40]:
n_cuda.unsqueeze(-1) - n_pt

tensor([[[[ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-1.1921e-07],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-1.1921e-07],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-5.9605e-08],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-4.7684e-07],
          [-3.5763e-07],
          [-7.1526e-07],
          [-7.1526e-07],
          [-4.7684e-07],
          [-3.5763e-07],
          [-3.5763e-07],
          [-4.7684e-07],
          [-2.3842e-07],
          [-5.0068e-06],
          [-6.1989e-06],
          [-5.9605e-07],
          [ 2.3842e-07],
          [ 0.0000e+00],
          [-3.5763e-07],
          [ 5.9605e-07],
          [ 1.5497e-06],
          [ 1.3113e-06],
          [ 9.5367e-07],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-1.9073e-06],
          [ 4.5300e-06],
          [ 2.5034e-06],
          [ 1.0729e-06],


In [14]:
m_cuda.unsqueeze(-1) - m_pt

tensor([[[[ 0.0000e+00],
          [ 2.9802e-08],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 5.9605e-08],
          [ 0.0000e+00],
          [-5.9605e-08],
          [ 0.0000e+00],
          [ 5.9605e-08],
          [ 1.1921e-07],
          [ 2.3842e-07],
          [ 0.0000e+00],
          [ 5.9605e-08],
          [ 0.0000e+00],
          [-1.7881e-07],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-6.5565e-07],
          [-8.3447e-07],
          [ 0.0000e+00],
          [-1.6093e-06],
          [ 0.0000e+00],
          [-1.6093e-06],
          [-2.2650e-06],
          [ 0.0000e+00],
          [-1.6093e-06],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-6.5565e-07],
          [ 5.9605e-07],
          [ 0.0000e+00],
          [-5.9605e-07],
          [ 5.9605e-07],
          [ 0.0000e+00],
          [ 2.3842e-07],
          [ 5.9605e-07],
          [ 1.9073e-06],
          [ 2.1458e-06],
          [ 2.3842e-06],


In [15]:
matD_cuda.shape

torch.Size([1, 1, 64, 64])

In [16]:
matD_cuda[:, :, :10, :10]

tensor([[[[ 3.5158e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,  0.0000e+00,  0.0000e+00],
          [ 3.8321e-02, -1.4171e+00,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,  0.0000e+00,  0.0000e+00],
          [-2.7494e-01, -1.7304e+00, -8.3489e-01,        -inf,        -inf,        -inf,        -inf,        -inf,  0.0000e+00,  0.0000e+00],
          [-5.8820e-01, -2.0437e+00, -1.1481e+00,  3.0957e-01,        -inf,        -inf,        -inf,        -inf,  0.0000e+00,  0.0000e+00],
          [-9.0146e-01, -2.3569e+00, -1.4614e+00, -3.6912e-03, -8.1536e-01,        -inf,        -inf,        -inf,  0.0000e+00,  0.0000e+00],
          [-1.2147e+00, -2.6702e+00, -1.7747e+00, -3.1695e-01, -1.1286e+00,  7.2678e-01,        -inf,        -inf,  0.0000e+00,  0.0000e+00],
          [-1.5280e+00, -2.9834e+00, -2.0879e+00, -6.3021e-01, -1.4419e+00,  4.1351e-01, -4.4421e-01,        -inf,  0.0000e+00,  0.0000e+00],
      

In [17]:
matD_pt

tensor([[[[ 3.5158e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf],
          [ 3.8321e-02, -1.4171e+00,        -inf,        -inf,        -inf,        -inf,        -inf,        -in

In [18]:
(matD_cuda - matD_pt)[:, :, :32, :10]

tensor([[[[ 0.0000e+00,         nan,         nan,         nan,         nan,         nan,         nan,         nan,         inf,         inf],
          [ 2.9802e-08,  0.0000e+00,         nan,         nan,         nan,         nan,         nan,         nan,         inf,         inf],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,         nan,         nan,         nan,         nan,         nan,         inf,         inf],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,         nan,         nan,         nan,         nan,         inf,         inf],
          [ 1.1921e-07,  2.3842e-07,  0.0000e+00,  5.9605e-08,  0.0000e+00,         nan,         nan,         nan,         inf,         inf],
          [ 1.1921e-07,  2.3842e-07,  1.1921e-07,  1.1921e-07,  0.0000e+00,  0.0000e+00,         nan,         nan,         inf,         inf],
          [ 0.0000e+00,  0.0000e+00,  2.3842e-07,  1.1921e-07,  0.0000e+00, -5.9605e-08,  0.0000e+00,         nan,         inf,         inf],
      

In [19]:
matD_pt[:, :, :10, :10]

tensor([[[[ 3.5158e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf],
          [ 3.8321e-02, -1.4171e+00,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf],
          [-2.7494e-01, -1.7304e+00, -8.3489e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf],
          [-5.8820e-01, -2.0437e+00, -1.1481e+00,  3.0957e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf],
          [-9.0146e-01, -2.3569e+00, -1.4614e+00, -3.6913e-03, -8.1536e-01,        -inf,        -inf,        -inf,        -inf,        -inf],
          [-1.2147e+00, -2.6702e+00, -1.7747e+00, -3.1695e-01, -1.1286e+00,  7.2678e-01,        -inf,        -inf,        -inf,        -inf],
          [-1.5280e+00, -2.9834e+00, -2.0879e+00, -6.3021e-01, -1.4419e+00,  4.1351e-01, -4.4421e-01,        -inf,        -inf,        -inf],
      